## Практическое задание к уроку 7 (Сверточные нейронные сети для анализа текста)

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# data_path = '/content/drive/MyDrive/Colab Notebooks/geekbrains natural language processing/homework7'
data_path = './'

In [3]:
!pip install stop_words
!pip install pymorphy2

### Задание

### Берем отызывы за лето (из архива с материалами или предыдущего занятия)

In [4]:
import pandas as pd
import os

In [5]:
df = pd.read_excel(os.path.join(data_path, 'reviews.xls'))
df.head(10)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,У меня работает все четко. В отличии от банком...,2017-08-14
9,5,Очень все хорошо👍,2017-08-14


### Препроцессинг

In [6]:
max_words = 200
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [7]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [8]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df['Content'] = df['Content'].apply(preprocess_text)
df['Rating'] = df['Rating'].apply(lambda x: f'{x}')

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Rating', 'Date']), df['Rating'], test_size=0.33, random_state=42)

In [10]:
train_corpus = " ".join(X_train["Content"])
train_corpus = train_corpus.lower()

In [11]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /Users/tombelov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
tokens_filtered = [word for word in tokens if word.isalnum()]
tokens_filtered[:10]

['наконецтый',
 'исправить',
 'чушь',
 'снеоргинальный',
 'прошивка',
 'приложение',
 'удобно',
 'пользоваться',
 'удобно',
 'использование']

In [13]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [14]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
vocabulary

{'приложение': 1,
 'удобно': 2,
 'работать': 3,
 'удобный': 4,
 'отлично': 5,
 'нравиться': 6,
 'хороший': 7,
 'отличный': 8,
 'телефон': 9,
 'супер': 10,
 'быстро': 11,
 'обновление': 12,
 'пароль': 13,
 'мочь': 14,
 'пользоваться': 15,
 'антивирус': 16,
 'банк': 17,
 'вход': 18,
 'устраивать': 19,
 'сбербанк': 20,
 'раз': 21,
 'прошивка': 22,
 'карта': 23,
 'проблема': 24,
 'рута': 25,
 'программа': 26,
 'ошибка': 27,
 'разработчик': 28,
 'сделать': 29,
 'приходиться': 30,
 'вводить': 31,
 'перевод': 32,
 'счёт': 33,
 'писать': 34,
 'норма': 35,
 'деньга': 36,
 'довольный': 37,
 'около': 38,
 'постоянно': 39,
 'нормально': 40,
 'код': 41,
 'исправить': 42,
 'смс': 43,
 'платёж': 44,
 'понятно': 45,
 'последний': 46,
 'функция': 47,
 'зайти': 48,
 'свой': 49,
 'вылетать': 50,
 'мобильный': 51,
 'стать': 52,
 'шаблон': 53,
 'приходить': 54,
 'возможность': 55,
 'право': 56,
 'делать': 57,
 'иня': 58,
 'проверка': 59,
 'класс': 60,
 'установить': 61,
 'root': 62,
 'заходить': 63,
 '5': 

In [15]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []

    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen - len(result))

    return padding + result[-maxlen:]

In [16]:
X_train = np.asarray([text_to_sequence(text, max_len) for text in X_train["Content"]], dtype=np.int32)
X_test = np.asarray([text_to_sequence(text, max_len) for text in X_test["Content"]], dtype=np.int32)

In [17]:
X_train, X_test

(array([[  0,   0,   0, ...,   1,   2,  15],
        [  0,   0,   0, ...,   0,   2, 181],
        [  0,   0,   0, ...,   0,   0,   5],
        ...,
        [  0,   0,   0, ..., 164,  27,  84],
        [  0,   0,   0, ...,   0,   0,  20],
        [  0,   0,   0, ...,   0, 113,   5]], dtype=int32),
 array([[ 0,  0,  0, ...,  0,  0,  5],
        [ 0,  0,  0, ...,  0,  6,  2],
        [ 0,  0,  0, ...,  1, 14, 48],
        ...,
        [ 0,  0,  0, ...,  1,  2, 11],
        [ 0,  0,  0, ..., 15,  1, 42],
        [ 0,  0,  0, ..., 32,  1, 91]], dtype=int32))

In [18]:
import numpy as np
import keras
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping

In [19]:
y_train.shape

(13841,)

In [20]:
num_classes = len(df['Rating'].value_counts()) + 1
num_classes

6

In [21]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

1. Создаем и учим модель с обычным Embedding

In [22]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [23]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [24]:
!pip install tensorflow==2.5.0

In [25]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
25/25 [==============================] - 11s 59ms/step - loss: 1.5644 - accuracy: 0.6370 - val_loss: 1.0197 - val_accuracy: 0.7011


In [26]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

14/14 [==============================] - 0s 7ms/step - loss: 0.9983 - accuracy: 0.7125


Test score: 0.9983162879943848
Test accuracy: 0.7125256657600403


In [27]:
results = model.predict(X_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 0s 6ms/step



2. Рассмотреть 2-а варианта сеточек

2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

2. Создаем и учим модель с предзагруженным Embedding

грузим с сайта https://rusvectores.org/ru/  
модель ruwikiruscorpora_tokens_elmo_1024_2019

In [28]:
!pip install --upgrade simple_elmo

In [29]:
from simple_elmo import ElmoModel
model_elmo = ElmoModel()
model_elmo.load(os.path.join(data_path, 'model'))

2021-07-29 20:46:12,219 : INFO : Loading model from ./model...
2021-07-29 20:46:12,222 : INFO : We will cache the vocabulary of 100 tokens.
/Users/tombelov/opt/miniconda3/lib/python3.9/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/Users/tombelov/opt/miniconda3/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


'The model is now loaded.'

In [30]:
vectors = model_elmo.get_elmo_vectors(df['Content'])

2021-07-29 20:46:19,637 : INFO : Warming up ELMo on 32 sentences...
2021-07-29 20:46:23,359 : INFO : Warming up finished.
2021-07-29 20:46:23,371 : INFO : Texts in the current batch: 32
2021-07-29 20:46:26,470 : INFO : Texts in the current batch: 32
2021-07-29 20:46:31,171 : INFO : Texts in the current batch: 32
2021-07-29 20:46:34,475 : INFO : Texts in the current batch: 32
2021-07-29 20:46:38,364 : INFO : Texts in the current batch: 32
2021-07-29 20:46:43,736 : INFO : Texts in the current batch: 32
2021-07-29 20:46:49,922 : INFO : Texts in the current batch: 32
2021-07-29 20:46:51,815 : INFO : Texts in the current batch: 32
2021-07-29 20:46:55,528 : INFO : Texts in the current batch: 32
2021-07-29 20:47:02,287 : INFO : Texts in the current batch: 32
2021-07-29 20:47:13,449 : INFO : Texts in the current batch: 32
2021-07-29 20:47:20,950 : INFO : Texts in the current batch: 32
2021-07-29 20:47:34,303 : INFO : Texts in the current batch: 32
2021-07-29 20:47:44,941 : INFO : Texts in the 

Увы, отваливаюсь по RAM, на colab аналогично - загрузить векторы, скачанные с сайта нет возможности.  
Прошу приложить pickle с моделью или предобученным Embedding слоем, ну или поднобные пошаговые инструкции, как можно загрузить эти векторы. 